In [17]:
#only need to run this once even if you have closed the notebook
#pip install yahoo_fin #need to run this before you use yahoo_fin 
import sys
!{sys.executable} -m pip install yahoo_fin
!{sys.executable} -m pip install pandas_datareader
!{sys.executable} -m pip install currencyconverter
!{sys.executable} -m pip install yfinance
#!pip3 install --upgrade pip
#!pip install requests_html

In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.stats as stats
import matplotlib.mlab as mlab
import seaborn as sns
from yahoo_fin import stock_info as si
import datetime 
import pandas_datareader as dr
import matplotlib.mlab as mlab
import seaborn as sns
from currency_converter import CurrencyConverter
import yfinance as yf
from bs4 import BeautifulSoup as bs #not used here but could be used if you wanted to scrape the web for info
import requests
import urllib.request
from html.parser import HTMLParser
import re
from statistics import mode, StatisticsError

**Preliminary Functions**

In [19]:
def flatten(x):
    result = []
    for el in x:
        if hasattr(el, "__iter__") and not isinstance(el, str):
            result.extend(flatten(el))
        else:
            result.append(el)
    return result

In [20]:
c = CurrencyConverter()

In [21]:
def list_to_string(s): 
  
    # initialization of string to "" 
    new = "" 
  
    # traverse in the string  
    for x in s: 
        new += x  
  
    # return string  
    return new 

In [22]:
#goals for exit multiple

#weighted_average [Frank]

#putting it all together - find_multiple(ticker) [Everyone]

#incorporating find_multiple into dcf function, 
#argument called exit_multiple = False [Everyone]

In [80]:
def get_Industry(ticker):
    #colins code
    ticker_profile_url = "https://finance.yahoo.com/quote/" + ticker + "/profile?p=" + ticker
    page = requests.get(ticker_profile_url)
    soup = bs(page.text, 'html.parser')
    ticker_profile = [x.text for x in soup.find_all('span', attrs={"class":"Fw(600)"})]
    try:
        ticker_sector = ticker_profile[1]
    except IndexError:
        print("Error: ticker does not have a listed sector. Unable to find comparables")
        exit(1)
    return ticker_sector

In [24]:
#returns the currency a stock's financial's are denoted in
def get_currency(tick):
    url = f'https://finance.yahoo.com/quote/{tick}?p={tick}&.tsrc=fin-srch'
    fpy = urllib.request.urlopen(url)
    mbyte = fpy.read()

    myst = mbyte.decode("utf8")
    fpy.close()
    
    var = []
    counter = 0
    vary = []
    
    for i in range(0,len(myst)):
        if myst[i]=='f':
            if myst[i+1]=='i':
                if myst[i+2]=='n':
                    if myst[i+3]=='a':
                        if myst[i+4]=='n':
                            if myst[i+5]=='c':
                                if myst[i+6]=='i':
                                    if myst[i+7]=='a':
                                        if myst[i+8]=='l':
                                            if myst[i+9]=='C':
                                                if myst[i+10]=='u':
                                                    if myst[i+11]=='r':
                                                        if myst[i+12]=='r':
                                                            if myst[i+13]=='e':
                                                                if myst[i+14]=='n':
                                                                    if myst[i+15]=='c':
                                                                        if myst[i+16]=='y':
                                                                            for j in range(20,23):
                                                                                if myst[i+j]=='}':
                                                                                    break
                                                                                elif myst[i+j]=='"':
                                                                                    pass
                                                                                elif myst[i+j]==',':
                                                                                    pass
                                                                                else: 
                                                                                    var.append(myst[i+j])
    vary = var[:3]         
    currency = list_to_string(vary)
    return currency

In [25]:
#use c.convert to convert an (amount, quoted currency, base currency)
def currency_convert(amount, ticker):
    try:
        return c.convert(amount,get_currency(ticker),'USD')
    except ValueError: 
        return amount

In [26]:
multipliers = {'M':10000000, 'B':1000000000, 'T':1000000000000}

def string_to_int(string):
    if string[-1].isdigit(): # check if no suffix
        return int(string)
    mult = multipliers[string[-1]] # look up suffix to get multiplier
     # convert number to float, multiply by multiplier, then make int
    return int(float(string[:-1]) * mult)

In [27]:
def get_numshares(ticker):
  mkt_cap = string_to_int(si.get_quote_table(ticker)["Market Cap"])
  price = si.get_live_price(ticker)
  return mkt_cap/price

In [28]:
#fits a gaussian to a histogram where n = bins 
def normdist(mean,stdev,n):
  return 1/(sigma * np.sqrt(2 * np.pi)) * np.exp( - (n - mu)**2 / (2 * sigma**2))

In [29]:
def error_in_the_mean(stdev,iterations):
  return sigma/(iterations^2)

In [30]:
#you can use this interval for bull / bear price analysis
def confidence_95(mean, stderror, iterations): #assumes 95% confidence for now, can have it return exact later
  x = stderror/np.sqrt(iterations)
  z_val = 1.96 #95% confidence
  lowerbound = mean - x*z_val
  upperbound = mean + x*z_val
  return (lowerbound,upperbound)

In [31]:
def confidence_99(mean,stderror,iterations):
  x = stderror/np.sqrt(iterations)
  z_val = 2.576 #99% confidence
  lowerbound = mean - x*z_val
  upperbound = mean + x*z_val
  return (lowerbound,upperbound)

In [32]:
#useful in data vis of pandas dataframes
def format_row(styler, formatter):
    for row, row_formatter in formatter.items():
        row_num = styler.index.get_loc(row)

        for col_num in range(len(styler.columns)):
            styler._display_funcs[(row_num, col_num)] = row_formatter
    return styler

In [33]:
def format_table(dataframe):
    pd.set_option('precision',4)
    out = dataframe.style.set_table_styles(
    [{'selector': 'th',
      'props': [('background', '#020D78'), 
                ('color', 'white'),
                ('font-family', 'helvetica')]},

     {'selector': 'td',
      'props': [('font-family', 'helvetica')]},

     {'selector': 'tr:nth-of-type(odd)',
      'props': [('background', '#D9DCF8')]}, 

     {'selector': 'tr:nth-of-type(even)',
      'props': [('background', 'white')]},

     {'selector': 'tr:hover',
      'props': [('background-color', '#D5E7D2')]},

     {'selector': 'th:hover', 
      'props': [('background-color', '#A8BCA5')]},

     {'selector': 'th',
      'props': [('text-align','left')]},

      {'selector': 'tr',
      'props': [('text-align','right')]},
     
     {'selector': 'th:nth-of-type(1)',
     'props': [('background-color','white'),
              ('color', 'black'),
              ('font-style', 'normal'),
              ('font-weight','normal')]}
     ])
    return out

In [34]:
#use this function to create a dataframe indexed by years in a dcf that shows the growth of a line item margin over time
def change(delta, ind):
    null = pd.Series(index=ind, dtype='float64')
    null[ind[0]]=1
    for i in range(1,len(ind)):
        null[i]=1+delta*i
    return null

In [35]:
#this is for cosmetics, if you want to display the percentage instead of use in a calculation
def percent_change(delta, ind): 
    null = pd.Series(index=ind)
    null[ind[0]]=0
    for i in range(1,len(ind)):
        null[i]=delta*i
    return null

In [36]:
def get_statements(ticker,prin=False):
    income_statement = si.get_income_statement(ticker)
    income_statement = income_statement.apply(pd.to_numeric, errors='coerce')
    income_statement = income_statement.replace(np.nan,0)

    balance_sheet = si.get_balance_sheet(ticker)
    balance_sheet = balance_sheet.apply (pd.to_numeric, errors='coerce')
    balance_sheet = balance_sheet.replace(np.nan,0)

    cashflow_statement = si.get_cash_flow(ticker)
    cashflow_statement = cashflow_statement.apply (pd.to_numeric, errors='coerce')
    cashflow_statement = cashflow_statement.replace(np.nan,0)

    if prin:
        print(income_statement, balance_sheet, cashflow_statement)
    return income_statement, balance_sheet, cashflow_statement

In [37]:
#Returns a firm's 3 financial statements
def get_coerced_statements(ticker, prin=False):
    income_statement = si.get_income_statement(ticker)
    income_statement = income_statement.apply(pd.to_numeric, errors='coerce')
    income_statement = income_statement.replace(np.nan,0)
 
    balance_sheet = si.get_balance_sheet(ticker)
    balance_sheet = balance_sheet.apply (pd.to_numeric, errors='coerce')
    balance_sheet = balance_sheet.replace(np.nan,0)
 
    cashflow_statement = si.get_cash_flow(ticker)
    cashflow_statement = cashflow_statement.apply (pd.to_numeric, errors='coerce')
    cashflow_statement = cashflow_statement.replace(np.nan,0)
    
    if prin:
        print(income_statement, balance_sheet, cashflow_statement)
    
    return (income_statement, balance_sheet, cashflow_statement)

In [38]:
get_coerced_statements('msft', prin=True)

endDate                              2021-06-30    2020-06-30    2019-06-30  \
Breakdown                                                                     
researchDevelopment                2.071600e+10  1.926900e+10  1.687600e+10   
effectOfAccountingCharges          0.000000e+00  0.000000e+00  0.000000e+00   
incomeBeforeTax                    7.110200e+10  5.303600e+10  4.368800e+10   
minorityInterest                   0.000000e+00  0.000000e+00  0.000000e+00   
netIncome                          6.127100e+10  4.428100e+10  3.924000e+10   
sellingGeneralAdministrative       2.522400e+10  2.452300e+10  2.309800e+10   
grossProfit                        1.158560e+11  9.693700e+10  8.293300e+10   
ebit                               6.991600e+10  5.314500e+10  4.295900e+10   
operatingIncome                    6.991600e+10  5.314500e+10  4.295900e+10   
otherOperatingExpenses             0.000000e+00  0.000000e+00  0.000000e+00   
interestExpense                   -2.330000e+09 -2.5

(endDate                              2021-06-30    2020-06-30    2019-06-30  \
 Breakdown                                                                     
 researchDevelopment                2.071600e+10  1.926900e+10  1.687600e+10   
 effectOfAccountingCharges          0.000000e+00  0.000000e+00  0.000000e+00   
 incomeBeforeTax                    7.110200e+10  5.303600e+10  4.368800e+10   
 minorityInterest                   0.000000e+00  0.000000e+00  0.000000e+00   
 netIncome                          6.127100e+10  4.428100e+10  3.924000e+10   
 sellingGeneralAdministrative       2.522400e+10  2.452300e+10  2.309800e+10   
 grossProfit                        1.158560e+11  9.693700e+10  8.293300e+10   
 ebit                               6.991600e+10  5.314500e+10  4.295900e+10   
 operatingIncome                    6.991600e+10  5.314500e+10  4.295900e+10   
 otherOperatingExpenses             0.000000e+00  0.000000e+00  0.000000e+00   
 interestExpense                   -2.33

In [39]:
#Returns a firm's income statement
def get_coerced_statement(ticker, statemen='income', prin=False):

    if statemen == 'income':

        statement = si.get_income_statement(ticker)
        statement = statement.apply(pd.to_numeric, errors='coerce')
        statement = statement.replace(np.nan,0)
        
    elif statemen == 'cashflow':
        
        statement = si.get_cash_flow(ticker)
        statement = statement.apply (pd.to_numeric, errors='coerce')
        statement = statement.replace(np.nan,0)
        
    elif statemen == 'balancesheet':
        
        statement = si.get_balance_sheet(ticker)
        statement = statement.apply (pd.to_numeric, errors='coerce')
        statement = statement.replace(np.nan,0)
        
    else: 
        
        statement = si.get_income_statement(ticker)
        statement = statement.apply(pd.to_numeric, errors='coerce')
        statement = statement.replace(np.nan,0)
        
    if prin:
        print(statement)

    return statement

In [40]:
#adjusts past prices for inflation
def get_normprice(past_price, start,end):
    inflation_data = dr.DataReader(['CPIAUCNS'], 'fred', start, end)
    start_CPI = inflation_data['CPIAUCNS'][0]
    cum_inf = []
    current_price = past_price
    for i in range (1,len(inflation_data)):
        cum_inf.append(((inflation_data['CPIAUCNS'][i] - inflation_data['CPIAUCNS'][i-1])/inflation_data['CPIAUCNS'][i-1])+1)
    for i in range(0,len(cum_inf)):
        current_price = current_price*cum_inf[i]
    return current_price

In [41]:
def get_rfr(term=10):
   
  #Josiah Sherk jossherk@umich.edu, Nicholas Frank frankn@umich.edu, Harrison Zeng hyzeng@umich.edu
 
  current_date = datetime.date.today()
  past_date = current_date-datetime.timedelta(days=100)
  if term == 1:
    Treasury = dr.DataReader(['DGS1'], 'fred', past_date, current_date)
  elif term == 2:
    Treasury = dr.DataReader(['DGS2'], 'fred', past_date, current_date)
  elif term == 2:
    Treasury = dr.DataReader(['DGS3'], 'fred', past_date, current_date)
  elif term == 5:
    Treasury = dr.DataReader(['DGS5'], 'fred', past_date, current_date)
  elif term == 10:
    Treasury = dr.DataReader(['DGS10'], 'fred', past_date, current_date)
  elif term == 20:
    Treasury = dr.DataReader(['DGS20'], 'fred', past_date, current_date)
  elif term == 30:
    Treasury = dr.DataReader(['DGS30'], 'fred', past_date, current_date)
  else:
    Treasury = dr.DataReader(['DGS10'], 'fred', past_date, current_date)
  RiskFreeRate = float(Treasury.iloc[-1])/100

  return RiskFreeRate

In [42]:
def get_NominalGrowth(USA=True):
    
    if not USA: 
        raise TypeError('Cannot find GDP for foriegn country on FRED')
    else: 
        current_date = datetime.date.today()
        future_date = current_date+datetime.timedelta(days=1000)
        GDP_forecast = dr.DataReader(['GDPC1CTM'], 'fred', current_date, future_date)
    return GDP_forecast['GDPC1CTM'][1]*(1/100)

In [43]:
get_NominalGrowth()

0.0235

In [44]:
get_rfr(term=10)

0.0144

In [45]:
np.mean([get_NominalGrowth(),get_rfr(term=10)])

0.01895

In [46]:
def get_revgrowth2(ticker):
#automate rev growth for DCF
#calculating avgerage data growth rate
  # revtotal = 0

  # income_statement = si.get_earnings(ticker)
  # income_statement = si.get_earnings(ticker)
  #get_income_statement(ticker, False)
  # income_statement = income_statement.apply(pd.to_numeric, errors='coerce')
  # income_statement = income_statement.replace(np.nan,0)
  #Compile the revenue by year into an array so that it can be cleaned
  quarterly_revenue = [ 22917000, 38668000, 19980000, 17911000, 15304000, 14139000, 11807000, 16908000]
  # fill quarterly_revenue
  # for x in range(number_of_years*4):
  #   quarterly_revenue.append() 

  # make a second array called CMA: start at i = 0, np.mean(i,i+1,i+2,i+3), ends when i+3 < length of revenue_by_Q
  cma = []
  for i in range(len(quarterly_revenue)-3):
    cma.append(np.mean([quarterly_revenue[i], quarterly_revenue[i+1], quarterly_revenue[i+2], quarterly_revenue[i+3]]))
  print(cma)

  ssi = []
  for i in range(len(cma)):
    ssi.append((quarterly_revenue[i+1]/cma[i])*100)
  print(ssi)

  typ_season = [0, 0, 0, 0]
  len_ts = [0, 0, 0, 0]
  for i in range(len(ssi)):
    i_real = i + 1
    typ_season[i_real%4] += ssi[i]
    len_ts[i_real%4] += 1
  for i in range(len(typ_season)):
    typ_season[i] = typ_season[i]/len_ts[i]
  # print(typ_season)

  deseasonalized = []
  for i in range(len(quarterly_revenue)):
    deseasonalized.append((quarterly_revenue[i]/typ_season[i%4])*100)
  print(deseasonalized)

  # return deseasonalized[-1] + deseasonalized[-2] + deseasonalized[-3] + deseasonalized[-4]

  

  # To make the SSI: start at j = 0, take (revenue_by_Q[j+1] /CMA[j])/100, ends when j < length CMA
  

  # print(income_statement)
  # print(income_statement["quarterly_revenue_earnings"])
  # number_of_years = 8 #can be changed
  # revenue_by_year = []
  # for x in range(number_of_years): #populates revenue_by_year
  #   revenue_by_year.append([income_statement.loc["totalRevenue"][x]])
  # #This should remove any outliers and produce a "clean" array of the income 
  # clean_income_array = remove_outlier(revenue_by_year)
  # print(revenue_by_year)
  # # Now, we would pass it through a loop in which it will calculate the revenue growth between intial year and initial year +1 and
  # # then totaling it, giving us rev total
  # for x in range(len(clean_income_array)):
  #   if (x < (len(clean_income_array)-1)):
  #     revtotal += (clean_income_array[x]-clean_income_array[x+1])/clean_income_array[x+1]
  # # print("the rev total is", revtotal)
  # finalrevgrowth= (1/len(clean_income_array))*revtotal
  # print("the final revgrowth is", finalrevgrowth)
  # If it is more than the cap (6%) then we will use the cap value. else we will average out the total revenue growth
  # return finalrevgrowth


In [47]:
def get_revgrowth1(ticker):
#automate rev growth for DCF
#calculating avgerage data growth rate 
#return (1/4)*((income_statement.loc["totalRevenue"][0]-income_statement.loc["totalRevenue"][3])/income_statement.loc["totalRevenue"][3])#previous growth rate from data
    beta= (si.get_quote_table(ticker)["Beta (5Y Monthly)"])
    current_date = datetime.date.today()
    future_date = current_date+datetime.timedelta(days=900)
    inf = dr.DataReader(['T10YIE'], 'fred', current_date, future_date)['T10YIE'][0] *0.01
    rfr = get_rfr(term=10)

    EMR = 0.06
    
    revgrowth = inf + beta*( EMR - inf)
    return revgrowth

In [48]:
def rev_growth3(ticker):
    #growth in net income
    #retention ratio*ROE
    #
    return 0

In [49]:
current_date = datetime.date.today()
future_date = current_date+datetime.timedelta(days=1000)
GDP_forecast = dr.DataReader(['GDPC1CTM'], 'fred', current_date, future_date)
print(GDP_forecast)

            GDPC1CTM
DATE                
2022-01-01      3.95
2023-01-01      2.35
2024-01-01      2.10


In [50]:
GDP_forecast['GDPC1CTM'][1]

2.35

In [51]:
def get_marketreturn(ticker):
    today = datetime.datetime.today()
    SP500_list = []
    for i in range (0,5):
            start = today - datetime.timedelta(days = 365*i)
            end = start - datetime.timedelta(days = 365)

            SP500 = dr.DataReader(['sp500'], 'fred', end, start)
            SP500.dropna(inplace = True)
            SP500YearlyReturn = (SP500['sp500'].iloc[-1]/ SP500['sp500'].iloc[0])-1
            SP500_list.append(SP500YearlyReturn)

    return np.mean(SP500_list)

In [52]:
def yearfrac():
    x1=(datetime.date.today())
    x2=(datetime.date(datetime.date.today().year, 12, 31))
    frac = (x2-x1)/(datetime.timedelta(days=365))
    return (x2-x1)/(datetime.timedelta(days=365))

In [53]:
def get_debt(ticker):
    stats_sheet = si.get_stats(ticker)


    return pd.DataFrame([stats_sheet])

In [54]:
#Calculate Cost of Equity
def get_costofequity(ticker, term = 10):
    '''
    start_date = datetime.date.today()-datetime.timedelta(days=365)  
    end_date = datetime.date.today()-datetime.timedelta(days=365)
    df = dr.get_data_yahoo(ticker, start_date, end_date)
    base_price = float((df['High'] + df['Low'])/2)
    divtable = dr.DataReader(ticker, 'yahoo-actions')
    
    base_div = float(divtable['value'][5])
    current_div = float(divtable['value'][0])
    Dividend Growth = (Dt/Dt-1) – 1
    div_growth = ((current_div - base_div)/base_div)
    div per share next year / current price + div growth 
    Re1 = (Dt / Pt-1) + ((Dt-Dt-1/Dt-1)) – 1
    CostofEquity1 = (current_div/base_price) + div_growth
    if no dividends re-route, if dividends, average it out
    '''
    
    #Calculate Market Return
    MarketReturn = get_marketreturn(ticker)

    #Calculate Risk Free Rate
    RiskFreeRate = get_rfr(term)
    #Pull Beta
    StockBeta = (si.get_quote_table(ticker)["Beta (5Y Monthly)"])
    #Capital Asset Pricing Model
    CostofEquity = RiskFreeRate + (StockBeta)*0.06 #0.06 = avg annual s&p reeturn
    
    #CostofEquity = (CostofEquity1 + CostofEquity2)/2

    return CostofEquity

In [55]:
#Calculate Firm Credit Spread
def get_creditspread(ticker):
    
  #Nicholas Frank frankn@umich.edu
  
  income_statement = si.get_income_statement(ticker)
  income_statement = income_statement.apply(pd.to_numeric, errors='coerce') 
  income_statement = income_statement.replace(np.nan,0)
  
  balance_sheet = si.get_balance_sheet(ticker)
  balance_sheet = balance_sheet.apply (pd.to_numeric, errors='coerce')
  balance_sheet = balance_sheet.replace(np.nan,0)
  
  cashflow_statement = si.get_cash_flow(ticker)
  cashflow_statement = cashflow_statement.apply (pd.to_numeric, errors='coerce')
  cashflow_statement = cashflow_statement.replace(np.nan,0)
  C = .05
  InterestCoverageRatio = (-1)*income_statement.loc["ebit"][0]/income_statement.loc["interestExpense"][0]

  #Credit Spreads from Damodaran for companies with MarketCap > $5 billion
  CreditSpread = 0.0156 + C
  if 8.5 < InterestCoverageRatio <= 100000:
    CreditSpread = 0.0063 + C
  if 6.5 < InterestCoverageRatio <= 8.499999:
    CreditSpread = 0.0078 + C 
  if 5.5 < InterestCoverageRatio <= 6.499999:
    CreditSpread = 0.0980 + C
  if 4.25 < InterestCoverageRatio <= 5.499999:
    CreditSpread = 0.0108 + C
  if 3 < InterestCoverageRatio <= 4.2499999:
    CreditSpread = 0.0122 + C
  if 2.5 < InterestCoverageRatio <= 2.99999:
    CreditSpread = 0.0156 + C
  if 2.25 < InterestCoverageRatio <= 2.499999:
    CreditSpread = 0.0200 + C
  if 2 < InterestCoverageRatio <= 2.249999:
    CreditSpread = 0.0240 + C
  if 1.75 < InterestCoverageRatio <= 1.99999:
    CreditSpread = 0.0351 + C
  if 1.5 < InterestCoverageRatio <= 1.749999:
    CreditSpread = 0.0421 + C
  if 1.25 < InterestCoverageRatio <= 1.499999:
    CreditSpread = 0.0515 + C
  if 0.8 < InterestCoverageRatio <= 1.249999:
    CreditSpread = 0.0820 + C
  if 0.65 < InterestCoverageRatio <= 0.799999:
    CreditSpread = 0.0864 + C
  if 0.2 < InterestCoverageRatio <= 0.6499999:
    CreditSpread = 0.1134 + C
  if -100000 < InterestCoverageRatio <= 0.199999:
    CreditSpread = 0.1512 + C

  return CreditSpread

In [56]:
def weighted_avg(data,outlier_threshold=1,weightings = False,rounding = False):
    el = np.array(data)
    mean = np.mean(el, axis=0)
    sd = np.std(el, axis=0)
    
    if len(el)<=2:
        if rounding:
            return round(np.mean(el),2)
        else:
            return np.mean(el)
    else:
        outlier_attributes = [[],[],[]] #outlier, index, type
        Upper_limit = mean + outlier_threshold * sd
        Lower_limit = mean - outlier_threshold * sd
        for i in range (0,len(el)): 
            if (el[i] <= Lower_limit):
                outlier_attributes[0].append(el[i])
                outlier_attributes[1].append(i)
                outlier_attributes[2].append('lower')
                el[i] = Lower_limit
            elif((el[i] >= Upper_limit)):
                outlier_attributes[0].append(el[i])
                outlier_attributes[1].append(i)
                outlier_attributes[2].append('upper')
                el[i] = Upper_limit

        weight_avg = np.mean(el)
        outliers = outlier_attributes[0]
        outlier_ind = outlier_attributes[1]
        outlier_thresh = outlier_attributes[2]
        num_outliers = len(outliers)
        num_vals = len(data)
        thresh = []
        alpha = []

        #calculation of alpha for weighting
        for i in range(0,len(outliers)):
            if outlier_thresh[i]=='upper':
                thresh.append(Upper_limit)
                alpha.append(thresh[i]/outliers[i])
            else: 
                thresh.append(Lower_limit)
                alpha.append(thresh[i]/outliers[i])

        residual_weighting = (num_outliers/num_vals)*(1-sum(alpha))
        incr_residual_weighting = residual_weighting/(num_vals-num_outliers)
        clean_weight = []

        for i in range(0,(num_vals-num_outliers)):
            clean_weight.append(1/num_vals+incr_residual_weighting)

        adj_weight = []

        for i in range(0,len(outliers)):
            adj_weight.append((1/num_vals)*alpha[i])

        weights = clean_weight + adj_weight

        if not weightings:
            if rounding:
                return round(np.mean(el),2)
            else:
                return np.mean(el)
        else:
            return weights

In [57]:
def find_Multiple(industry='TotalMarket(withoutfinancials)', financials = False):
    if financials and industry == 'TotalMarket(withoutfinancials)': 
        industry = 'TotalMarket'
    EVEBIDTA = pd.read_html('http://pages.stern.nyu.edu/~adamodar/New_Home_Page/datafile/vebitda.html')
    df = EVEBIDTA[0]
    #Any NAN value will be changed to 10 EV / EBIDTA
    df.drop(df.columns[[2,4,5,6,7,8,9]], axis=1, inplace=True)
    for i in range(0,len(df)):
        sentence = str(df[0][i])
        df[0][i]=''.join(sentence.split())
    df[3].fillna(10, inplace=True)
    #try: 
    try:
        
        for i in range(0,len(df)):
            if df[0][i] == industry:
                return df[3][i]
                break
            elif i == len(df):
                return 10
                break
                
    except KeyError:
        return 10

In [58]:
def table_contains(user_string='Total Market'):
    industry_list = []
    user_strings = []
    
    if 'Semiconductors' in user_string:
        user_string = 'Semiconductor'

    if '—' in user_string:
        user_strings = user_string.split('—')
        
    else:
        user_strings = user_string.split()


    for i in range(0,len(user_strings)):  
        if ' ' in user_strings[i]:
            user_strings[i] = user_strings[i].split()

    user_strings = flatten(user_strings)
    
    EVEBIDTA = pd.read_html('http://pages.stern.nyu.edu/~adamodar/New_Home_Page/datafile/vebitda.html')
    df = EVEBIDTA[0]
    #Any NAN value will be changed to 10 EV / EBIDTA
    df.drop(df.columns[[2,4,5,6,7,8,9]], axis=1, inplace=True)
    for i in range(0,len(df)):
        sentence = str(df[0][i])
        df[0][i]=''.join(sentence.split())
    df[3].fillna(10, inplace=True)
    
    for strings in user_strings:
        
        for i in range(0,len(df)):
            
            if strings in df[0][i]:
                industry_list.append(df[0][i])
         
    try:
        return mode(industry_list)
    except StatisticsError:
        indlist = []
        [indlist.append(x) for x in industry_list if x not in indlist] 
        return indlist

In [59]:
def get_ExitMultiple(ticker):
    user_strin = get_Industry(ticker)
    multiples = []
    multiple = 0
    string_list = table_contains(user_strin)
    if len(string_list[0])==1:
        string_list = [string_list]
    for i in range (0,len(string_list)):
        multiples.append(find_Multiple(string_list[i]))
        multiples_float = [float(mult) for mult in multiples]
        multiple = weighted_avg(multiples_float,rounding=True)
    return multiple

**WACC**

In [77]:
def get_wacc(ticker, term=10):
   
  #Nicholas Frank frankn@umich.edu, Harrison Zeng hyzeng@umich.edu

  income_statement = si.get_income_statement(ticker)
  income_statement = income_statement.apply(pd.to_numeric, errors='coerce')
  income_statement = income_statement.replace(np.nan,0)
 
  balance_sheet = si.get_balance_sheet(ticker)
  balance_sheet = balance_sheet.apply (pd.to_numeric, errors='coerce')
  balance_sheet = balance_sheet.replace(np.nan,0)
   
  totalDebt =  balance_sheet.loc["longTermDebt"][0] + balance_sheet.loc["shortLongTermDebt"][0] + balance_sheet.loc['cash'][0]
  try:
    totalDebt
  except KeyError:
    totalDebt = balance_sheet.loc["longTermDebt"][0] + balance_sheet.loc['cash'][0]
   
  try:
    totalDebtLastYear = balance_sheet.loc["longTermDebt"][1] + balance_sheet.loc["shortLongTermDebt"][1] + balance_sheet.loc['cash'][1]
  except KeyError:
    totalDebtLastYear = balance_sheet.loc["longTermDebt"][1] + balance_sheet.loc['cash'][1]
  #two-yr moving debt average
  bookDebt = ((totalDebt + totalDebtLastYear)/2)
  #Market Value is Total Debt and Equity
  MarketCap = string_to_int(si.get_quote_table(ticker)["Market Cap"])
  #EV = D + V
  #EnterpriseValue = netDebt + balance_sheet.loc["commonStock"][0] + income_statement.loc["minorityInterest"][0] + PreferredStock #minority = non-controlled
  EnterpriseValue = bookDebt + MarketCap
  tax_rate = ((income_statement.loc["incomeTaxExpense"][0]/income_statement.loc["incomeBeforeTax"][0]) + (income_statement.loc["incomeTaxExpense"][1]/income_statement.loc["incomeBeforeTax"][1])) / 2

  #Calculate Risk Free Rate
  RiskFreeRate = get_rfr(term)

  #Calculate Cost of Debt Pre Tax
  CostofDebt_PreTax_GuruFocus = ((-1)*income_statement.loc["interestExpense"][0]) / bookDebt
 
  #Calculate Cost of Equity
  CostofEquity = get_costofequity(ticker, term)

  #wacc = (E/V)*Ce + (D/V)*Cd*(Tax Shield)
  wacc = (MarketCap/EnterpriseValue)*CostofEquity + (bookDebt/EnterpriseValue)*CostofDebt_PreTax_GuruFocus*(1-tax_rate)
  #wacc is currently 6.7% for aapl, hoping to be around 8.24% for aapl
  return wacc

In [78]:
get_wacc('msft')

0.06484275248710814

**Revenue Growth**

In [64]:
def get_revgrowth(ticker):
#automate rev growth for DCF
#calculating avgerage data growth rate 
  return (1/4)*((income_statement.loc["totalRevenue"][0]-income_statement.loc["totalRevenue"][3])/income_statement.loc["totalRevenue"][3])#previous growth rate from data


**Terminal Growth**

In [65]:
def get_termgrowth(ticker):
  #limit of rev growth to infinity? for DCF
    return termgrowth

**DDM (Dividend Discount Model)**

In [66]:
def ddm(ticker, inv_term, new_eq):
    
    #implied dividend growth rate = ROE * Payout Ratio
    earnings = si.get_income_statement(ticker).loc['netIncome'][0]
    mean_eq = 0.5*(si.get_balance_sheet('msft').loc['totalStockholderEquity'][0]
    + si.get_balance_sheet(ticker).loc['totalStockholderEquity'][1])
    
    roe = earnings/mean_eq
    payout_ratio = -si.get_cash_flow(ticker).loc['dividendsPaid'][0]/earnings
    growth_vector = []
    r = get_costofequity(ticker, inv_term)
    
    divtable = dr.DataReader(ticker, 'yahoo-actions')
        
    #base_div = float(divtable['value'][len(divtable)-1])
    current_div = float(divtable['value'][0])
    
    #Dividend Growth = (Dt/Dt-1) – 1
    #g = (((current_div - base_div)/base_div))/5
    #for i in range(1,len(divtable)-1):
       # growth_vector.append(float(divtable['value'][i-1])-float(divtable['value'][i]))
    #g = 4*np.mean(growth_vector)
    
    g = roe * payout_ratio
    
    #growing infinite geometric series formula
    price = 0
    for i in range(0,5):
        price += currency_convert(-new_eq[i],ticker)/(1+r)**i
        
    price += (currency_convert(-new_eq[5],ticker)/(r-g))/(1+r)**6
   
    return price/get_numshares(ticker)

**Discounted Cash Flow Model (DCF)**

In [67]:
def basecase_dcf(ticker,wacc,rev_growth=0.05, term_growth=0.029,rev_grow=0.0,cogs_grow=0.0, sga_grow=0.0,
                 rd_grow=0.0,capex_grow=0.0,wacc_step=0.005,term_growth_step=0.005, format_price = True, 
                 table=True, ExitMultiple=False, div_discount=False, upperlower_bound=False):
    
  #Josiah Sherk jossherk@umich.edu, Joshua Hartman hartjosh@umich.edu, Nicholas Frank frankn@umich.edu

  #new and improved dcf function, now corrects for foriegn currency & exit multiple
  #note: term_growth cannot exceed 2.9% (GDP growth of US) for any company

    income_statement = si.get_income_statement(ticker)
    income_statement = income_statement.apply (pd.to_numeric, errors='coerce')
    income_statement = income_statement.replace(np.nan,0)
 
    balance_sheet = si.get_balance_sheet(ticker)
    balance_sheet = balance_sheet.apply (pd.to_numeric, errors='coerce')
    balance_sheet = balance_sheet.replace(np.nan,0)
 
    cashflow_statement = si.get_cash_flow(ticker)
    cashflow_statement = cashflow_statement.apply (pd.to_numeric, errors='coerce')
    cashflow_statement = cashflow_statement.replace(np.nan,0)

    years = ['2021', '2022', '2023', '2024', '2025','2026']
    revenue = pd.Series(index=years, dtype='float64')
    
    #customize dcf
    rev_gr = change(rev_grow,years)
    cogs_growth = change(cogs_grow,years)
    sga_growth = change(sga_grow,years)
    rd_growth = change(rd_grow,years)
    capex_growth = change(capex_grow,years)
    

    #begin change_nwc calculation
    #define lists

    days_sales = []
    proj_ar = []
    avg_ar = []
    avg_inv = []
    proj_inv = []
    days_inventory = []
    proj_excess_assets = []
    proj_currentassets = []

    days_ap = []
    avg_ap = []
    proj_ap = []
    proj_excess_liabilities = []
    proj_currentliabilities = []

    proj_nwc = []
    nwc = []

    #projected revenue
    current_revenue = income_statement.loc["totalRevenue"][0]
    avgdata_growthrate = rev_growth 
    revenue[years[0]] = current_revenue*(1+avgdata_growthrate)

    for year in range(1, 6):
        revenue[year] = revenue[year-1] * (1 + avgdata_growthrate)
    
    revenue = revenue*rev_gr

    #current assets
    try:  
        historic_rev = income_statement.loc['totalRevenue'] #will be slightly off
    except KeyError:
        historic_rev = 0*revenue #will produce NaN if this error occurs

    try:
        ar = balance_sheet.loc['netReceivables'] #accounts receivable preferred, days of sales may be different for ar
    except KeyError:
        ar = 0*historic_rev

    try:
        inventory = balance_sheet.loc['inventory']
    except KeyError:
        inventory = 0*historic_rev

    try:
        historic_cogs = income_statement.loc['costOfRevenue']
    except KeyError:
        historic_cogs = 0*historic_rev
    
    try:
        current_remainder = np.median((balance_sheet.loc['totalCurrentAssets']-ar-inventory)/historic_rev)
    except KeyError: 
        current_remainder = 0*historic_rev

    #current liabilities
    try:
        ap = balance_sheet.loc['accountsPayable']
    except KeyError:
        ap = 0*historic_rev

    try:
        current_liabilities_remainder = np.median((balance_sheet.loc['totalCurrentLiabilities']-ap)/historic_rev)
    except KeyError:
        current_liabilities_remainder = 0*historic_rev

    try:
        proj_cogs_margin = income_statement.loc['costOfRevenue'][0]/current_revenue*cogs_growth
    except KeyError:
        proj_cogs_margin = 0*revenue

    proj_cogs = proj_cogs_margin*revenue
    gross_profit = revenue-proj_cogs #same as gross_margin*revenue

    #for year 0 = 2019. can make each value's margin increase by some constant in each period

    for i in range(0,len(historic_rev)-1):
        avg_ar.append(1/2*(ar[len(historic_rev)-1-i] + ar[len(historic_rev)-2-i])) #starts at year 2016
        days_sales.append(365*(avg_ar[i]/historic_rev[len(historic_rev)-2-i])) #starts at year 2016
        avg_inv.append(1/2*(inventory[len(historic_rev)-1-i] + inventory[len(historic_rev)-2-i])) #starts at year 2016
        days_inventory.append(365*(avg_inv[i]/historic_cogs[len(historic_rev)-2-i])) #starts at year 2016
        avg_ap.append((1/2*(ap[len(historic_rev)-1-i] + ap[len(historic_rev)-2-i])))
        days_ap.append(avg_ap[i]/historic_cogs[len(historic_rev)-2-i])

    med_days_sales = np.median(days_sales)
    med_days_inv = np.median(days_inventory)
    med_days_ap = np.median(days_ap)

    for i in range(0,6):
        proj_ar.append((med_days_sales/365)*revenue[i])
        proj_inv.append((med_days_inv/365)*proj_cogs[i])
        proj_excess_assets.append(current_remainder*revenue[i])
        proj_ap.append((med_days_ap/365)*proj_cogs[i])
        proj_excess_liabilities.append(current_liabilities_remainder*revenue[i])

    for i in range (0,len(proj_ar)):
        proj_currentassets.append( proj_ar[i] + proj_inv[i] + proj_excess_assets[i])
        proj_currentliabilities.append(proj_ap[i] + proj_excess_liabilities[i])
        proj_nwc.append(proj_currentassets[i]-proj_currentliabilities[i])

    #ppe forecast
    historical_ppe = balance_sheet.loc['propertyPlantEquipment']
    
    avg_ppe = ((1/len(historical_ppe)-1))*np.sum(historical_ppe/historic_rev)
        
    proj_ppe = avg_ppe*revenue
    
    #depreciation forecast
    
    historical_da = cashflow_statement.loc['depreciation']

    avg_da = ((1/len(historical_da)-1))*np.sum(historical_da/historical_ppe)
     
    avg_ppelist = []
    
    avg_ppe1 = 0.5*(proj_ppe[0]+historical_ppe[0])
    
    avg_ppelist.append(avg_ppe1)
    
    for year in range(1,6):
        avg_ppelist.append(0.5*(proj_ppe[year]+proj_ppe[year-1]))
    
    #this will give the depreciation for each projected period
    proj_da = []
    for i in range(0,len(avg_ppelist)):
        proj_da.append(avg_da*avg_ppelist[i])
    
    #solve for capex 
    #capex = da_t + (pp_t-ppe_t-1)
    
    proj_capex = []
    
    proj_capex.append(proj_da[0]+(proj_ppe[0]-historical_ppe[0]))
  
    for i in range(1,6):
        proj_capex.append(proj_da[i]+(proj_ppe[i]-proj_ppe[i-1]))
        
    capex = pd.Series(proj_capex, index = years, dtype='float64')
        
    #long term investments/revenue
    
    historical_investments = balance_sheet.loc['longTermInvestments']
    
    avg_inv = ((1/len(historical_investments)-1))*np.sum(historical_investments/historic_rev)
        
    proj_inv = avg_inv*revenue
    
    #intangibles/revenue
    
    try:
        
        historical_intangibles = balance_sheet.loc['intangibleAssets']

        avg_intangibles = ((1/len(historical_intangibles)-1))*np.sum(historical_intangibles/historic_rev)
        
        proj_intangibles = avg_intangibles*revenue
        
    except KeyError:
    
        proj_intangibles = 0*revenue
        
    #deferred taxes - add in the future

    #proj_total assets
    
    proj_totalassets = []
    
    for i in range(0,6):
        proj_totalassets.append(proj_ppe[i]+proj_intangibles[i]+proj_inv[i]+proj_currentassets[i])
    
    #long term debt forecast - can set this as a fixed ratio to total assets (total assets forecast)
    
    historical_ltd = balance_sheet.loc['longTermDebt']
    
    avg_ltd = ((1/len(historical_ltd)-1))*np.sum(historical_ltd/balance_sheet.loc['totalAssets'])
    
    proj_ltd = []
    
    for i in range(0,len(proj_totalassets)):  
        proj_ltd.append(avg_ltd*proj_totalassets[i])
    
    #plug number for equity financing
    
    new_equity = [] #positive if equity is issued, negative if repurchased
    
    for i in range(0,6):
        new_equity.append(proj_totalassets[i]-(proj_ltd[i]+proj_currentliabilities[i]))
        
    change_nwc = []
    current_nwc = (balance_sheet.loc['totalCurrentAssets'][1]-balance_sheet.loc['totalCurrentLiabilities'][1])
    change_nwc.append((proj_nwc[0] - current_nwc))
    #change_nwc.append(0)

    for i in range(1, 6):
        change_nwc.append(proj_nwc[i] - proj_nwc[i-1])

    change_nwc = pd.Series(change_nwc, index=years, dtype='float64')
    
    #loop through change_nwc and add ppe, subtract LTD to each for equity 
    
    #end change_nwc calculation

    try:
        sga_margin = income_statement.loc["sellingGeneralAdministrative"][0]/current_revenue*sga_growth
    except KeyError:
        sga_margin = 0*revenue
    try:
        rd_margin = income_statement.loc["researchDevelopment"][0]/current_revenue*rd_growth
    except KeyError:
        rd_margin = 0*revenue

    op_exp = sga_margin*revenue + rd_margin*revenue
  
    op_income = gross_profit - op_exp

    #nonop_other_income_margin = 
    #nonop_exp_margin 
    #ebit = op_income + nonop_income_margin*revenue-nonop_exp_margin*revenue
  
    da = pd.Series(proj_da, index =years, dtype='float64')
  
    #ebitda = ebit - da

    #interest_expense_margin 
    try:
        tax_rate = income_statement.loc["incomeTaxExpense"][0]/income_statement.loc['incomeBeforeTax'][0]
    except KeyError:
        tax_rate = 0.2 #default corporate tax 20%

    nopat = op_income*(1-tax_rate) #roughly the same as ebiat

    #if want to get super precise could build out B/S and do relevant "days of" for current assets/liabilities * I/S items

    #deferred_taxes_margin --> deferred_taxes_margin*revenue would be added to ufcf if available
    
    
    ufcf = nopat + da - change_nwc - proj_capex 
    
    base_pv_ufcf = (((yearfrac()*ufcf[0])*(1 / (1 + wacc)) ** yearfrac())+sum(ufcf[1:] * [(1 / (1 + wacc)) ** (i+yearfrac()) for i in range (1,6)]))
        
    if upperlower_bound:
        upper_pv_ufcf = ((yearfrac()*ufcf[0])*(1 / (1 + (wacc-wacc_step)) ** 
                        yearfrac())+sum(ufcf[1:] * [(1 / (1 + (wacc-wacc_step))) **
                        (i+yearfrac()) for i in range (1,6)]))
        lower_pv_ufcf = (((yearfrac()*ufcf[0])*(1 / (1 + (wacc+wacc_step))) ** yearfrac())+sum(ufcf[1:] * [(1 / (1 + (wacc+wacc_step))) ** 
                        (i+yearfrac()) for i in range (1,6)]))
                         
        upper_termval = ((ufcf[5] * (1 + (term_growth+term_growth_step)))/((wacc-wacc_step) - (term_growth+term_growth_step)))
        base_termval = ((ufcf[5] * (1 + term_growth))/(wacc - term_growth))
        lower_termval = ((ufcf[5] * (1 + (term_growth-term_growth_step)))/((wacc+wacc_step) - (term_growth-term_growth_step)))               
        
        
        if not ExitMultiple:
            upper_termval = ((ufcf[5] * (1 + (term_growth+term_growth_step)))/((wacc-wacc_step) - (term_growth+term_growth_step)))
            base_termval = ((ufcf[5] * (1 + term_growth))/(wacc - term_growth))
            lowertermval = ((ufcf[5] * (1 + (term_growth-term_growth_step)))/((wacc+wacc_step) - (term_growth-term_growth_step)))               
        else:
            ebitda = op_income[5]+da[5] #+nonop_income[5]-nonop_expenses[5]
            base_termval = get_ExitMultiple(ticker)*ebitda
        
        upper_pv_termval = base_termval/((1+(wacc-wacc_step))**(6+yearfrac()))
        base_pv_termval = base_termval/((1+wacc)**(6+yearfrac()))
        lower_pv_termval = base_termval/((1+(wacc+wacc_step))**(6+yearfrac()))
        
        upper_ev = upper_pv_ufcf + upper_pv_termval
        base_ev = base_pv_ufcf + base_pv_termval
        lower_ev = lower_pv_ufcf + lower_pv_termval

        num_shares = get_numshares(ticker)

        try:
            debt = balance_sheet.loc['longTermDebt'][0]
        except KeyError:
            try:
                debt = balance_sheet.loc['shortLongTermDebt'][0]
            except KeyError:
                debt = 0

        try:
            cash = balance_sheet.loc['cash'][0]
        except KeyError:
            cash = 0
        
        upper_share_price = currency_convert((((upper_ev - debt + cash)))/num_shares,ticker)
        base_share_price = currency_convert((((base_ev - debt + cash)))/num_shares,ticker)
        lower_share_price = currency_convert((((lower_ev - debt + cash)))/num_shares,ticker)
        
        print(f"The upper implied share price of ${ticker} is ${round(upper_share_price,2)}," +
              f" the base implied share price is ${round(base_share_price,2)}," +
              f"and the lower implied share price is ${round(lower_share_price,2)}.")

    
    if not ExitMultiple:
        base_termval = ((ufcf[5] * (1 + term_growth))/(wacc - term_growth))
    else:
        ebitda = op_income[5]+da[5] #+nonop_income[5]-nonop_expenses[5]
        base_termval = get_ExitMultiple(ticker)*ebitda
        
    base_pv_termval = base_termval/((1+wacc)**(6+yearfrac()))
    base_ev = base_pv_ufcf + base_pv_termval
      
    num_shares = get_numshares(ticker)

    try:
        debt = balance_sheet.loc['longTermDebt'][0]
    except KeyError:
        try:
            debt = balance_sheet.loc['shortLongTermDebt'][0]
        except KeyError:
            debt = 0

    try:
        cash = balance_sheet.loc['cash'][0]
    except KeyError:
        cash = 0

    base_share_price = currency_convert((((base_ev - debt + cash)))/num_shares,ticker)
    
    if not table:
        if format_price:
            return f"${round(base_share_price,2)}"
        else: 
            return base_share_price
    else:
        
        upside = ((base_share_price-si.get_live_price(ticker))/si.get_live_price(ticker)) #what % higher is your intrinsic price than live price
        downside = ((base_share_price-si.get_live_price(ticker))/si.get_live_price(ticker)) #what % higher is your intrinsic price than live price
        result = " downside"
        if (upside > 0):
            result = "n upside"

        
        output = pd.DataFrame([round(revenue/1000000,2),rev_growth*change(rev_grow,years),round(proj_cogs/1000000,2),
                               round(gross_profit/1000000,2),gross_profit/revenue,round(sga_margin*revenue/1000000,2), 
                               rd_margin*revenue/1000000,op_income/1000000,op_income/revenue,da/1000000,
                               round((capex*(revenue/revenue))/1000000, 2),(capex/revenue),round(change_nwc/1000000,2),
                               round(nopat/1000000,2),round(ufcf/1000000,2),round(new_equity*(revenue/revenue)/1000000,2)],
                            index=['Revenue','Revenue Growth','COGS','Gross Profit','Gross Margin','SG&A', 
                                   'R&D','Operating Income','Operating Margin','D&A','Capex','Capex % Revenue',
                                   'Change NWC','NOPAT','Unlevered FCF','New Equity'])

        #pd.set_option("precision",7)
        print(f"Sample Discounted Cash Flow of {ticker} in millions with WACC {round(wacc*100,2)}% and Terminal Growth {round(term_growth*100,2)}%")

        print(f"The implied share price of {ticker} is ${round(base_share_price,2)} with a{result} of {round(upside*100,2)}%")

        out1 = output.style.set_properties(**{'width': '100px'})
        out2 = out1.set_table_styles(
        [{'selector': 'th',
          'props': [('background', '#020D78'), 
                    ('color', 'white'),
                    ('font-family', 'helvetica')]},

         {'selector': 'td',
          'props': [('font-family', 'helvetica')]},

         {'selector': 'tr:nth-of-type(odd)',
          'props': [('background', '#D9DCF8')]}, 

         {'selector': 'tr:nth-of-type(even)',
          'props': [('background', 'white')]},

         {'selector': 'tr:hover',
          'props': [('background-color', '#D5E7D2')]},

         {'selector': 'th:hover', 
          'props': [('background-color', '#A8BCA5')]},

         {'selector': 'th',
          'props': [('text-align','left')]},

          {'selector': 'tr',
          'props': [('text-align','right')]},

         {'selector': 'th:nth-of-type(1)',
         'props': [('background-color','white'),
                  ('color', 'black'),
                  ('font-style', 'normal'),
                  ('font-weight','normal')]},

         {'selector': 'tr:nth-of-type(2)',
          'props': [('color','#606060'),
                   ('font-style','italic')]},

         {'selector': 'tr:nth-of-type(5)',
          'props': [('color','#606060'),
                   ('font-style','italic')]},

         {'selector': 'tr:nth-of-type(9)',
          'props': [('color','#606060'),
                   ('font-style','italic')]},

         {'selector': 'tr:nth-of-type(12)',
          'props': [('color','#606060'),
                   ('font-style','italic')]}
         ])

        formatters = {"Revenue": lambda x:f"{x:,.2f}","Revenue Growth": lambda x:f"{x:.2%}","COGS": lambda x:f"{x:,.2f}",
                      "Gross Profit": lambda x:f"{x:,.2f}", "Gross Margin": lambda x:f"{x:.2%}", "SG&A": lambda x:f"{x:,.2f}",
                     "R&D": lambda x:f"{x:,.2f}", "Operating Income": lambda x:f"{x:,.2f}","Operating Margin": lambda x:f"{x:.2%}",
                      "D&A": lambda x:f"{x:,.2f}", "Capex": lambda x:f"{x:,.2f}", "Capex % Revenue": lambda x:f"{x:.2%}",
                     "Change NWC": lambda x:f"{x:,.2f}", "NOPAT": lambda x:f"{x:,.2f}","Unlevered FCF": lambda x:f"{x:,.2f}", 
                      "New Equity": lambda x:f"{x:,.2f}"}
        styler = format_row(out2,formatters)
        if div_discount:
            print(f"The estimated price of {ticker} by the Dividend Discount Model is {round(ddm(ticker, 5, new_equity),2)}")
            
        return styler

In [68]:
#Josiah Sherk jossherk@umich.edu, Nicholas Frank frankn@umich.edu

#bluesky dcf has lower cogs grow, lower sga grow, higher rev growth, higher rev grow
def blue_sky_dcf(tick, weighted_acc, revenue_growth=0.05, terminal_growth=0, revenue_convex=0, cogs_delta=0, 
                 sga_delta=0, rd_delta=0, capex_delta=0, wacc_delta=0.005, term_growth_delta=0.005, 
                 format_share_price=True, table_grid=True, revenue_growth_step=0.005, revenue_convex_step = 0.005,
                 cogs_delta_step=0.005, sga_delta_step=0.005, rd_delta_step=0.000, capex_delta_step=0.000, 
                 Exit_Multiple=False, divdiscount=False, upperlowerbound=False):
    
    cogs_delta-=cogs_delta_step
    sga_delta-=sga_delta_step
    revenue_growth+=revenue_growth_step
    revenue_convex+=revenue_convex_step
    rd_delta-=rd_delta_step
    capex_delta-= capex_delta_step
    
    return basecase_dcf(tick, wacc=weighted_acc, rev_growth=revenue_growth, term_growth=terminal_growth, 
                        rev_grow=revenue_convex, cogs_grow=cogs_delta, sga_grow=sga_delta, rd_grow=rd_delta,
                        capex_grow=capex_delta, wacc_step=wacc_delta, term_growth_step=term_growth_delta, 
                        format_price=format_share_price, table=table_grid, ExitMultiple=Exit_Multiple, 
                        div_discount=divdiscount, upperlower_bound=upperlowerbound)

In [69]:
#Josiah Sherk jossherk@umich.edu, Nicholas Frank frankn@umich.edu

#greysky dcf has higher cogs grow, higher sga grow, lower rev growth, lower rev grow
def grey_sky_dcf(tick, weighted_acc, revenue_growth=0.05, terminal_growth=0, revenue_convex=0, cogs_delta=0, 
                 sga_delta=0, rd_delta=0, capex_delta=0, wacc_delta=0.005, term_growth_delta=0.005, 
                 format_share_price=True, table_grid=True, revenue_growth_step=0.005, revenue_convex_step = 0.005,
                 cogs_delta_step=0.005, sga_delta_step=0.005, rd_delta_step=0.000, capex_delta_step=0.000, 
                 Exit_Multiple=False, divdiscount=False, upperlowerbound=False):
    
    cogs_delta+=cogs_delta_step
    sga_delta+=sga_delta_step
    revenue_growth-=revenue_growth_step
    revenue_convex-=revenue_convex_step
    rd_delta+=rd_delta_step
    capex_delta+= capex_delta_step
    
    return basecase_dcf(tick, wacc=weighted_acc, rev_growth=revenue_growth, term_growth=terminal_growth, 
                        rev_grow=revenue_convex, cogs_grow=cogs_delta, sga_grow=sga_delta, rd_grow=rd_delta,
                        capex_grow=capex_delta, wacc_step=wacc_delta, term_growth_step=term_growth_delta, 
                        format_price=format_share_price, table=table_grid, ExitMultiple=Exit_Multiple, 
                        div_discount=divdiscount, upperlower_bound=upperlowerbound)

In [91]:
blue_sky_dcf('MSFT', 0.06484275248710814, revenue_growth=0.143, terminal_growth = 0.03, format_share_price = True, table_grid=True, Exit_Multiple=True, divdiscount=True)

Error: ticker does not have a listed sector. Unable to find comparables


UnboundLocalError: local variable 'ticker_sector' referenced before assignment

In [81]:
basecase_dcf('MSFT', wacc = 0.06484275248710814, rev_growth=0.143, term_growth = 0.03, format_price = True, table=True, ExitMultiple=True, div_discount=True, upperlower_bound=True)

Error: ticker does not have a listed sector. Unable to find comparables


UnboundLocalError: local variable 'ticker_sector' referenced before assignment

In [ ]:
grey_sky_dcf('MSFT', wacc = 0.06484275248710814, rev_growth=0.143, format_share_price = True, table_grid=True, Exit_Multiple=True, divdiscount=True)

In [ ]:
#Josiah Sherk jossherk@umich.edu, Nicholas Frank frankn@umich.edu

def master_dcf(ticker, wacc, rev_growth=0.05, term_growth=0.029, rev_grow=0.0, cogs_grow=0.0,
                sga_grow=0.0,rd_grow=0.0,capex_grow=0.0, wacc_step=0.005, term_growth_step =0.005,
                format_price = True, table=True, ExitMultiple=False, div_discount=False, 
                upperlower_bound=False, basecase=False, bluesky=False, greysky=False):
    #monte carlo as outer loop. more defaults in master. argument list=long
    #or monte carlo as first if statement. need to fix up monte carlo before implementing
    if bluesky:
        blue_sky_dcf(ticker, weighted_acc=wacc, revenue_growth=rev_growth, terminal_growth=term_growth,
                    revenue_convex=rev_grow, cogs_delta=cogs_grow, sga_delta=sga_grow, rd_delta=rd_grow, 
                    capex_delta=capex_grow, wacc_delta=wacc_step, term_growth_delta=term_growth_step,
                    format_share_price=format_price, table_grid=table, Exit_Multiple=ExitMultiple, 
                    divdiscount=div_discount, upperlowerbound=upperlower_bound)
    if greysky:
        grey_sky_dcf(ticker, weighted_acc=wacc, revenue_growth=rev_growth, terminal_growth=term_growth,
                    revenue_convex=rev_grow, cogs_delta=cogs_grow, sga_delta=sga_grow, rd_delta=rd_grow, 
                    capex_delta=capex_grow, wacc_delta=wacc_step, term_growth_delta=term_growth_step,
                    format_share_price=format_price, table_grid=table, Exit_Multiple=ExitMultiple, 
                    divdiscount=div_discount, upperlowerbound=upperlower_bound)   
    if basecase:
        basecase_dcf(ticker, wacc, rev_growth, term_growth, rev_grow, cogs_grow,
                    sga_grow, rd_grow, capex_grow, wacc_step, term_growth_step,
                    format_price, table, ExitMultiple, div_discount, upperlower_bound)

    return None

In [ ]:
master_dcf('MSFT', wacc = 0.062, term_growth = 0.03, format_price = True, table=True, ExitMultiple=False, div_discount=False, upperlower_bound=True, basecase=True, bluesky=True,greysky=True)

In [88]:
basecase_dcf('MSFT', wacc = 0.062, term_growth = 0.03, format_price = True, table=True, ExitMultiple=True, div_discount=False, upperlower_bound=True)

Error: ticker does not have a listed sector. Unable to find comparables


UnboundLocalError: local variable 'ticker_sector' referenced before assignment

In [ ]:
basecase_dcf('MSFT', wacc = 0.06, rev_growth = 0.08, rev_grow = 0.03, cogs_grow = -0.03, sga_grow = 0.01, ExitMultiple=True, table=True)

In [ ]:
#for special cases and error handling, have optional boolean argument called "data_report" 
#that prints all assumptions made (i.e. the year each line item margin is pulled from or if 
#there were key errors encountered). Doesn't seem to like CLDX, pulls 0 for r&d

**Sensitivity Analysis**

In [73]:
#tablex and tabley are the dimensions of the table. defaults to wacc term_growth table. The 12th and 13th arguments allow user to input what they would like to see 
#reflected in the table. Consider the other arguments integer labels 0 to 12, 
#and enter integers for these arguments

#Instructions for Frank and Anika:

#Make this compatible with our current basecase_dcf function. This means replacing calls to dcf with basecase_dcf 
#and adding Exit EV/EBITDA as an option for an x / y value to be changed on the pivot table. Will need to add that 
#inside the for loops / conditional statements

def sensitivity_table(ticker,wacc1=0.08,rev_growth=0.05,termgr=0.029,
                      rev_convexity=0.0,cogs_growth=0.0,sga_growth=0.0,
                      rd_growth=0.0,capex_growth=0.0, exit_growth=0.0,step1=0.005,step2=0.005,tablex=5,tabley=5,
                      specify1=1,specify2=3):
    
    if specify1 == specify2:
        specify1 = 1
        specify2 = 3
        
    if specify1 == 1:
        base1 = round(wacc1,2)
        str1 = 'WACC'
    elif specify1 == 2:
        base1 = round(rev_growth,2)
        str1 = 'Rev Growth'
    elif specify1 == 3: 
        base1 = round(termgr,2)
        str1 = 'Term Growth'
    elif specify1 == 4:
        base1 = round(rev_convexity,2)
        str1 = 'Rev Convexity'
    elif specify1 == 5: 
        base1 = round(cogs_growth,2)
        str1 = 'COGS Growth'
    elif specify1 == 6: 
        base1 = round(sga_growth,2)
        str1 = 'SGA Growth'
    elif specify1 == 7:
        base1 = round(rd_growth,2)
        str1 = 'R&D Growth'
    elif specify1 == 8:
        base1 = round(capex_growth,2)
        str1 = 'Capex Growth'
    elif specify1 == 9:
        base1 = round(exit_growth,2)
        str1 = 'Exit EV/EBITDA'
    else:
        specify1 == 1
        base1 = round(wacc1,2)
        str1 = 'WACC'
        
    if specify2 == 1:
        base2 = round(wacc1,2)
        str2 = 'WACC'
    elif specify2 == 2:
        base2 = round(rev_growth,2)
        str2 = 'Rev Growth'
    elif specify2 == 3: 
        base2 = round(termgr,2)
        str2 = 'Term Growth'
    elif specify2 == 4:
        base2 = round(rev_convexity,2)
        str2 = 'Rev Convexity'
    elif specify2 == 5: 
        base2 = round(cogs_growth,2)
        str2 = 'COGS Growth'
    elif specify2 == 6: 
        base2 = round(sga_growth,2)
        str2 = 'SGA Growth'
    elif specify2 == 7:
        base2 = round(rd_growth,2)
        str2 = 'R&D Growth'
    elif specify2 == 8:
        base2 = round(capex_growth,2)
        str2 = 'Capex Growth'
    elif specify2 == 9:
        base2 = round(exit_growth,2)
        str2 = 'Exit EV/EBITDA'
    else:
        specify2 == 3
        base2 = round(termgr,2)
        str2 = 'Term Growth'
        
    list1 = []
    for i in range(0,tablex):
        list1.append((base1 - 2*step1) + i*step1)
        
    list2 = []
    for i in range(0,tabley):
        list2.append((base2 - 2*step2) + i*step2)
    
    sensitivity = {}
    for i in range(0,tablex):
        sensitivity[f"{round(list1[i]*100,4)}% {str1}"] = {}
    for i in range(0,tabley):    
        sensitivity[f"{round(list2[i]*100,4)}% {str2}"] = {}
        
    array = list(zip(list1, list2))
    
    for i in range(0,tablex):
        for j in range(0,tabley):
            if specify1 == 1:
                if specify2 == 2: 
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,array[j][0],array[i][1],termgr,
                        rev_convexity,cogs_growth,sga_growth,rd_growth,capex_growth,exit_growth,table=False)
                elif specify2 == 3:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,array[j][0],rev_growth,array[i][1],
                        rev_convexity,cogs_growth,sga_growth,rd_growth,capex_growth,exit_growth,table=False)
                elif specify2 == 4:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,array[j][0],rev_growth,termgr,
                        array[i][1],cogs_growth,sga_growth,rd_growth,capex_growth,exit_growth,table=False)
                elif specify2 == 5:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,array[j][0],rev_growth,termgr,
                       rev_convexity,array[i][1],sga_growth,rd_growth,capex_growth,exit_growth,table=False)
                elif specify2 == 6:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,array[j][0],rev_growth,termgr,
                       rev_convexity,cogs_growth,array[i][1],rd_growth,capex_growth,exit_growth,table=False)
                elif specify2 == 7:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,array[j][0],rev_growth,termgr,
                       rev_convexity,cogs_growth,sga_growth,array[i][1],capex_growth,exit_growth,table=False)
                elif specify2 == 8:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,array[j][0],rev_growth,termgr,
                       rev_convexity,cogs_growth,sga_growth,rd_growth,array[i][1],exit_growth,table=False)
                elif specify2 == 9:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,array[j][0],rev_growth,termgr,
                       rev_convexity,cogs_growth,sga_growth,rd_growth,capex_growth,array[i][1],table=False)
            elif specify1 == 2:
                if specify2 == 1:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,array[i][1],array[j][0],termgr,
                       rev_convexity,cogs_growth,sga_growth,rd_growth,capex_growth,exit_growth,table=False)
                elif specify2 == 3:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,array[j][0],array[i][1],
                       rev_convexity,cogs_growth,sga_growth,rd_growth,capex_growth,exit_growth,table=False)
                elif specify2 == 4:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,array[j][0],termgr,
                       array[i][1],cogs_growth,sga_growth,rd_growth,capex_growth,exit_growth,table=False)
                elif specify2 == 5:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,array[j][0],termgr,
                       rev_convexity,array[i][1],sga_growth,rd_growth,capex_growth,exit_growth,table=False)
                elif specify2 == 6:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,array[j][0],termgr,
                       rev_convexity,cogs_growth,array[i][1],rd_growth,capex_growth,exit_growth,table=False)
                elif specify2 == 7:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,array[j][0],termgr,
                       rev_convexity,cogs_growth,sga_growth,array[i][1],capex_growth,exit_growth,table=False)
                elif specify2 == 8:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,array[j][0],termgr,
                       rev_convexity,cogs_growth,sga_growth,rd_growth,array[i][1],exit_growth,table=False)
                elif specify2 == 9:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,array[j][0],termgr,
                       rev_convexity,cogs_growth,sga_growth,rd_growth,capex_growth,array[i][1],table=False)
            elif specify1 == 3:
                if specify2 == 1:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,array[i][1],rev_growth,array[j][0],
                       rev_convexity,cogs_growth,sga_growth,rd_growth,capex_growth,exit_growth,table=False)    
                elif specify2 == 2:  
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,array[i][1],array[j][0],
                       rev_convexity,cogs_growth,sga_growth,rd_growth,capex_growth,exit_growth,table=False)  
                elif specify2 == 4:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,rev_growth,array[j][0],
                       array[i][1],cogs_growth,sga_growth,rd_growth,capex_growth,exit_growth,table=False)
                elif specify2 == 5:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,rev_growth,array[j][0],
                       rev_convexity,array[i][1],sga_growth,rd_growth,capex_growth,exit_growth,table=False)
                elif specify2 == 6:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,rev_growth,array[j][0],
                       rev_convexity,cogs_growth,array[i][1],rd_growth,capex_growth,exit_growth,table=False)
                elif specify2 == 7:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,rev_growth,array[j][0],
                       rev_convexity,cogs_growth,sga_growth,array[i][1],capex_growth,exit_growth,table=False)
                elif specify2 == 8:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,rev_growth,array[j][0],
                       rev_convexity,cogs_growth,sga_growth,rd_growth,array[i][1],exit_growth,table=False)
                elif specify2 == 9:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,rev_growth,array[j][0],
                       rev_convexity,cogs_growth,sga_growth,rd_growth,capex_growth,array[i][1],table=False)
            elif specify1 == 4:
                if specify2 == 1:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,array[i][1],rev_growth,termgr,
                       array[j][0],cogs_growth,sga_growth,rd_growth,capex_growth,exit_growth,table=False)    
                elif specify2 == 2: 
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,array[i][1],termgr,
                       array[j][0],cogs_growth,sga_growth,rd_growth,capex_growth,exit_growth,table=False) 
                elif specify2 == 3:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,rev_growth,array[i][1],
                       array[j][0],cogs_growth,sga_growth,rd_growth,capex_growth,exit_growth,table=False)     
                elif specify2 == 5:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,rev_growth,termgr,
                       array[j][0],array[i][1],sga_growth,rd_growth,capex_growth,exit_growth,table=False) 
                elif specify2 == 6:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,rev_growth,termgr,
                       array[j][0],cogs_growth,array[i][1],rd_growth,capex_growth,exit_growth,table=False) 
                elif specify2 == 7:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,rev_growth,termgr,
                       array[j][0],cogs_growth,sga_growth,array[i][1],capex_growth,exit_growth,table=False) 
                elif specify2 == 8:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,rev_growth,termgr,
                       array[j][0],cogs_growth,sga_growth,rd_growth,array[i][1],exit_growth,table=False)
                elif specify2 == 9:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,rev_growth,termgr,
                       array[j][0],cogs_growth,sga_growth,rd_growth,capex_growth,array[i][1],table=False)  
            elif specify1 == 5:
                if specify2 == 1:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,array[i][1],rev_growth,termgr,
                       rev_convexity,array[j][0],sga_growth,rd_growth,capex_growth,exit_growth,table=False)     
                elif specify2 == 2: 
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,array[i][1],termgr,
                       rev_convexity,array[j][0],sga_growth,rd_growth,capex_growth,exit_growth,table=False) 
                elif specify2 == 3:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,rev_growth,array[i][1],
                       rev_convexity,array[j][0],sga_growth,rd_growth,capex_growth,exit_growth,table=False)
                elif specify2 == 4:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,rev_growth,termgr,
                       array[i][1],array[j][0],sga_growth,rd_growth,capex_growth,exit_growth,table=False)
                elif specify2 == 6:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,rev_growth,termgr,
                       rev_convexity,array[j][0],array[i][1],rd_growth,capex_growth,exit_growth,table=False)
                elif specify2 == 7:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,rev_growth,termgr,
                       rev_convexity,array[j][0],sga_growth,array[i][1],capex_growth,exit_growth,table=False)
                elif specify2 == 8:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,rev_growth,termgr,
                       rev_convexity,array[j][0],sga_growth,rd_growth,array[i][1],exit_growth,table=False)
                elif specify2 == 9:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,rev_growth,termgr,
                       rev_convexity,array[j][0],sga_growth,rd_growth,capex_growth,array[i][1],table=False)
            elif specify1 == 6:
                if specify2 == 1:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,array[i][1],rev_growth,termgr,
                       rev_convexity,cogs_growth,array[j][0],rd_growth,capex_growth,exit_growth,table=False)          
                elif specify2 == 2:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,array[i][1],termgr,
                       rev_convexity,cogs_growth,array[j][0],rd_growth,capex_growth,exit_growth,table=False) 
                elif specify2 == 3: 
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,rev_growth,array[i][1],
                       rev_convexity,cogs_growth,array[j][0],rd_growth,capex_growth,exit_growth,table=False) 
                elif specify2 == 4:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,rev_growth,termgr,
                       array[i][1],cogs_growth,array[j][0],rd_growth,capex_growth,exit_growth,table=False)
                elif specify2 == 5:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,rev_growth,termgr,
                       rev_convexity,array[i][1],array[j][0],rd_growth,capex_growth,exit_growth,table=False)
                elif specify2 == 7:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,rev_growth,termgr,
                       rev_convexity,cogs_growth,array[j][0],array[i][1],capex_growth,exit_growth,table=False)
                elif specify2 == 8:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,rev_growth,termgr,
                       rev_convexity,cogs_growth,array[j][0],rd_growth,array[i][1],exit_growth,table=False)
                elif specify2 == 9:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,rev_growth,termgr,
                       rev_convexity,cogs_growth,array[j][0],rd_growth,capex_growth,array[i][1],table=False)
            elif specify1 == 7:
                if specify2 == 1:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,array[i][1],rev_growth,termgr,
                       rev_convexity,cogs_growth,sga_growth,array[j][0],capex_growth,exit_growth,table=False)           
                elif specify2 == 2: 
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,array[i][1],termgr,
                       rev_convexity,cogs_growth,sga_growth,array[j][0],capex_growth,exit_growth,table=False)  
                elif specify2 == 3:  
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,rev_growth,array[i][1],
                       rev_convexity,cogs_growth,sga_growth,array[j][0],capex_growth,exit_growth,table=False)
                elif specify2 == 4:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,rev_growth,termgr,
                       array[i][1],cogs_growth,sga_growth,array[j][0],capex_growth,exit_growth,table=False)
                elif specify2 == 5:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,rev_growth,termgr,
                       rev_convexity,array[i][1],sga_growth,array[j][0],capex_growth,exit_growth,table=False)
                elif specify2 == 6:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,rev_growth,termgr,
                       rev_convexity,cogs_growth,array[i][1],array[j][0],capex_growth,exit_growth,table=False)
                elif specify2 == 8:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,rev_growth,termgr,
                       rev_convexity,cogs_growth,sga_growth,array[j][0],array[i][1],exit_growth,table=False)
                elif specify2 == 9:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,rev_growth,termgr,
                       rev_convexity,cogs_growth,sga_growth,array[j][0],capex_growth,array[i][1],table=False)
            elif specify1 == 8:
                if specify2 == 1:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,array[i][1],rev_growth,termgr,
                       rev_convexity,cogs_growth,sga_growth,rd_growth,array[j][0],exit_growth,table=False)           
                elif specify2 == 2: 
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,array[i][1],termgr,
                       rev_convexity,cogs_growth,sga_growth,rd_growth,array[j][0],exit_growth,table=False)  
                elif specify2 == 3:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,rev_growth,array[i][1],
                       rev_convexity,cogs_growth,sga_growth,rd_growth,array[j][0],exit_growth,table=False)
                elif specify2 == 4:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,rev_growth,termgr,
                       array[i][1],cogs_growth,sga_growth,rd_growth,array[j][0],exit_growth,table=False)
                elif specify2 == 5:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,rev_growth,termgr,
                       rev_convexity,array[i][1],sga_growth,rd_growth,array[j][0],exit_growth,table=False) 
                elif specify2 == 6:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,rev_growth,termgr,
                       rev_convexity,cogs_growth,array[i][1],rd_growth,array[j][0],exit_growth,table=False) 
                elif specify2 == 7:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,rev_growth,termgr,
                       rev_convexity,cogs_growth,sga_growth,array[i][1],array[j][0],exit_growth,table=False) 
                elif specify2 == 9:
                    sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,rev_growth,termgr,
                       rev_convexity,cogs_growth,sga_growth,rd_growth,array[j][0],array[i][1],table=False) 
            elif specify1 == 9:
                if specify2 == 1:
                  sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,array[i][1],rev_growth,termgr,
                       rev_convexity,cogs_growth,sga_growth,rd_growth,capex_growth,array[j][0],table=False)
                if specify2 == 2:
                  sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,array[i][1],termgr,
                       rev_convexity,cogs_growth,sga_growth,rd_growth,capex_growth,array[j][0],table=False)
                if specify2 == 3:
                  sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,rev_growth,array[i][1],
                       rev_convexity,cogs_growth,sga_growth,rd_growth,capex_growth,array[j][0],table=False)
                if specify2 == 4:
                  sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,rev_growth,termgr,
                       array[i][1],cogs_growth,sga_growth,rd_growth,capex_growth,array[j][0],table=False)
                if specify2 == 5:
                  sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,rev_growth,termgr,
                       rev_convexity,array[i][1],sga_growth,rd_growth,capex_growth,array[j][0],table=False)
                if specify2 == 6:
                  sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,rev_growth,termgr,
                       rev_convexity,cogs_growth,array[i][1],rd_growth,capex_growth,array[j][0],table=False)
                if specify2 == 7:
                  sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,rev_growth,termgr,
                       rev_convexity,cogs_growth,sga_growth,array[i][1],capex_growth,array[j][0],table=False)
                if specify2 == 8:
                  sensitivity[f"{round(list2[i]*100,4)}% {str2}"][f"{round(list1[j]*100,4)}% {str1}"] = basecase_dcf(ticker,wacc1,rev_growth,termgr,
                       rev_convexity,cogs_growth,sga_growth,rd_growth,array[i][1],array[j][0],table=False)
    sensitivity_analysis = pd.DataFrame.from_dict(sensitivity,orient='columns').dropna(axis=1) 
    print(f"Sensitivity Analysis on {str1} and {str2} for {ticker}")
    table = format_table(sensitivity_analysis)
    return table

sensitivity_table('msft', 0.06)

In [86]:
sensitivity_table('MSFT',wacc1 = 0.06484275248710814,rev_growth=0.143)

Sensitivity Analysis on WACC and Term Growth for MSFT


,2.0% Term Growth,2.5% Term Growth,3.0% Term Growth,3.5% Term Growth,4.0% Term Growth
5.0% WACC,$548.61,$648.8,$799.08,$1049.55,$1550.5
5.5% WACC,$466.25,$536.11,$633.91,$780.62,$1025.12
6.0% WACC,$404.6,$455.75,$523.96,$619.44,$762.67
6.5% WACC,$356.75,$395.6,$445.55,$512.14,$605.37
7.0% WACC,$318.57,$348.91,$386.85,$435.62,$500.65


In [85]:
sensitivity_table('MSFT',wacc1=0.06484275248710814,rev_growth=0.143,termgr=0.029,
                      rev_convexity=0.0,cogs_growth=-0.01,sga_growth=0.0,
                      rd_growth=0.0,capex_growth=0.0,step1=0.005,step2=0.005,tablex=5,tabley=5,
                      specify1=2,specify2=5)

Sensitivity Analysis on Rev Growth and COGS Growth for MSFT


,-2.0% COGS Growth,-1.5% COGS Growth,-1.0% COGS Growth,-0.5% COGS Growth,0.0% COGS Growth
13.0% Rev Growth,$426.3,$419.59,$412.87,$413.0,$406.18
13.5% Rev Growth,$438.25,$431.36,$424.47,$424.62,$417.61
14.0% Rev Growth,$450.48,$443.41,$436.34,$436.5,$429.31
14.5% Rev Growth,$463.0,$455.74,$448.48,$448.66,$441.28
15.0% Rev Growth,$475.8,$468.36,$460.91,$461.1,$453.53


**Monte Carlo DCF**

In [96]:
#updated monte carlo with change_nwc and KeyError Handling
def mc_dcf(ticker, wacc=get_wacc(ticker,5), rev_growth=0.05, 
           term_growth=0.029, rev_grow=0.0, cogs_grow=0.0, 
           sga_grow=0.0, rd_grow=0.0, capex_grow=0.0, iterations=50000):

  #Josiah Sherk jossherk@umich.edu, Joshua Hartman hartjosh@umich.edu, Nicholas Frank frankn@umich.edu

  #inputs
  current_price = si.get_live_price(ticker)
  num_shares = string_to_int(si.get_quote_table(ticker)["Market Cap"])/current_price 
  
  #define lists
  days_sales = []
  proj_ar = []
  avg_ar = []
  avg_inv = []
  proj_inv = []
  days_inventory = []
  proj_excess_assets = []
  proj_currentassets = []

  days_ap = []
  avg_ap = []
  proj_ap = []
  proj_excess_liabilities = []
  proj_currentliabilities = []

  proj_nwc = []
  nwc = []

  #financial statement variables
  income_statement = si.get_income_statement(ticker)
  income_statement = income_statement.apply(pd.to_numeric, errors='coerce')
  income_statement = income_statement.replace(np.nan,0)

  balance_sheet = si.get_balance_sheet(ticker)  
  balance_sheet = balance_sheet.apply (pd.to_numeric, errors='coerce')  
  balance_sheet = balance_sheet.replace(np.nan,0)

  cashflow_statement = si.get_cash_flow(ticker) 
  cashflow_statement = cashflow_statement.apply (pd.to_numeric, errors='coerce')  
  cashflow_statement = cashflow_statement.replace(np.nan,0)
    
  #customize dcf
  years = ['2021', '2022', '2023', '2024', '2025','2026']
    
  rev_gr = change(rev_grow,years)
  cogs_gr = change(cogs_grow,years)
  sga_gr = change(sga_grow,years)
  rd_gr = change(rd_grow,years)
  capex_gr = change(capex_grow,years)

  #define debt / cash
  try:
    debt = balance_sheet.loc['longTermDebt'][0]
  except KeyError: 
    try:
      debt = balance_sheet.loc['shortLongTermDebt'][0]
    except KeyError:
      debt = 0

  try:
    cash = balance_sheet.loc['cash'][0]
  except KeyError:
    cash = 0

  #revenue
  revenue = pd.Series(index=years)
  revenue_growth_dist = np.random.normal(rev_growth, 0.01, size = (6,1)) 

  current_revenue = income_statement.loc["totalRevenue"][0]
  revenue[years[0]] = current_revenue #(1+revenue_growth_dist[0])

  #check line items for key errors and define variables 

  #current assets

  try:  
    historic_rev = income_statement.loc['totalRevenue'] #will be slightly off
  except KeyError:
    historic_rev = 0*revenue #will produce NaN if this error occurs

  try:
    ar = balance_sheet.loc['netReceivables'] #accounts receivable preferred, days of sales may be different for ar
  except KeyError:
    ar = 0*historic_rev

  try:
    inventory = balance_sheet.loc['inventory']
  except KeyError:
    inventory = 0*historic_rev

  try:
    current_inv = balance_sheet.loc['inventory'][0]
  except KeyError:
    current_inv = 0*revenue

  try:
    historic_cogs = income_statement.loc['costOfRevenue']
  except KeyError:
    historic_cogs = 0*historic_rev
    
  try:
    current_remainder = np.median((balance_sheet.loc['totalCurrentAssets']-ar-inventory)/historic_rev)
  except KeyError: 
    current_remainder = 0*historic_rev

  #current liabilities
  try:
    ap = balance_sheet.loc['accountsPayable']
  except KeyError:
    ap = 0*historic_rev

  try:
    current_liabilities_remainder = np.median((balance_sheet.loc['totalCurrentLiabilities']-ap)/historic_rev)
  except KeyError:
    current_liabilities_remainder = 0*historic_rev
  
  #line items for dcf (not change_nwc calculation)
  #change_nwc backward looking elements (no distribution)
  #median days of sales

  for i in range(0,len(historic_rev)-1):
    avg_ar.append(1/2*(ar[len(historic_rev)-1-i] + ar[len(historic_rev)-2-i])) #starts at year 2016
    days_sales.append(365*(avg_ar[i]/historic_rev[len(historic_rev)-2-i])) #starts at year 2016
    avg_inv.append(1/2*(inventory[len(historic_rev)-1-i] + inventory[len(historic_rev)-2-i])) #starts at year 2016
    days_inventory.append(365*(avg_inv[i]/historic_cogs[len(historic_rev)-2-i])) #starts at year 2016
    avg_ap.append((1/2*(ap[len(historic_rev)-1-i] + ap[len(historic_rev)-2-i])))
    days_ap.append(avg_ap[i]/historic_cogs[len(historic_rev)-2-i])

  med_days_sales = np.median(days_sales)
  med_days_inv = np.median(days_inventory)
  med_days_ap = np.median(days_ap)

  #***place for loop here
  target_price = []
  for i in range(0,iterations):
    for j in range(1, 6):
      if j <= len(revenue):
        revenue[j] = revenue[j - 1] * (1 + revenue_growth_dist[j-1])
      else:
        revenue[j-1] = revenue[j - 2] * (1 + revenue_growth_dist[j-1])
        
    revenue = revenue*rev_gr    

    #line items for dcf (not change_nwc calculation)

    try: 
      current_cogs = income_statement.loc['costOfRevenue'][0]
    except KeyError:
      current_cogs = 0
    
    try: 
      sga = income_statement.loc['sellingGeneralAdministrative'][0]
    except KeyError:
      sga = 0

    try: 
      rd = income_statement.loc['researchDevelopment'][0]
    except KeyError:
      rd = 0

    try:
      da = cashflow_statement.loc['depreciation'][0]*(revenue/revenue)
    except KeyError:
      da = 0

    try: 
      capex = cashflow_statement.loc['capitalExpenditures'][0]
    except KeyError:
      capex = 0

    try:
      tax_expense = income_statement.loc["incomeTaxExpense"][0]
    except KeyError:
      tax_expense = 0
    
    try:
      pretax_income = income_statement.loc['incomeBeforeTax'][0]
    except KeyError:
      pretax_income = 0

    try:
      tax_rate = tax_expense/pretax_income
    except ZeroDivisionError:
      tax_rate = 0.24  

    #define margin distributions for dcf
    projcogs_dist = np.random.normal(current_cogs/current_revenue,0.005,10*iterations)
    sga_dist = np.random.normal(sga/current_revenue,0.005,10*iterations)
    rd_dist = np.random.normal(rd/current_revenue,0.005,10*iterations)
    capex_dist = np.random.normal(-capex/current_revenue,0.005,10*iterations)
    projinv_dist = np.random.normal(current_inv/current_revenue,0.005,10*iterations)
    wacc_dist = np.random.normal(wacc,0.0001,10*iterations)

    #change_nwc calculation

    for k in range(0,5):
      proj_ar.append((med_days_sales/365)*revenue[k])
      proj_inv.append((med_days_inv/365)*projcogs_dist[i]*revenue[k])
      proj_excess_assets.append(current_remainder*revenue[k])
      proj_ap.append((med_days_ap/365)*projcogs_dist[i]*revenue[k])
      proj_excess_liabilities.append(current_liabilities_remainder*revenue[k])

    for k in range (0,5):
      proj_currentassets.append( proj_ar[k] + projinv_dist[i]*revenue[k] + proj_excess_assets[k])
      proj_currentliabilities.append(proj_ap[k] + proj_excess_liabilities[k])
      proj_nwc.append(proj_currentassets[k]-proj_currentliabilities[k])

    change_nwc = []
    current_nwc = (balance_sheet.loc['totalCurrentAssets'][1]-balance_sheet.loc['totalCurrentLiabilities'][1])
    change_nwc.append((proj_nwc[0] - current_nwc))
    change_nwc.append((proj_nwc[1] - proj_nwc[0]))

    for k in range(1, 5):
      change_nwc.append(proj_nwc[k] - proj_nwc[k-1])

    change_nwc = pd.Series(change_nwc, index=years)

    #end change_nwc calculation

    #ufcf calculation
    gross_profit = revenue - revenue*cogs_gr*projcogs_dist[i] 
    op_income = gross_profit - sga_dist[i]*sga_gr*revenue - rd_dist[i]*rd_gr*revenue
    nopat = op_income*(1-tax_rate)
    ufcf = nopat + da + change_nwc - capex_dist[i]*capex_gr*revenue

    base_termval = (ufcf[5] * (1 + term_growth)/(wacc_dist[i] - term_growth))
    pv_ufcf = (sum(ufcf[1:] * [(1 / (1 + wacc_dist[i])) ** j for j in range (0,5)]))
    pv_termval = (base_termval/((1+wacc_dist[i])**(6)))
    ev = pv_ufcf + pv_termval
    target_price.append((ev - debt + cash)/num_shares)

  (mu, sigma) = stats.norm.fit(target_price)
  max = np.max(target_price)

  result = (mu-si.get_live_price(ticker))/si.get_live_price(ticker)
  returns = "n upside"

  if (result <= 0):
    returns = " downside"   

  print(f"We are 95% confident the implied share price of {ticker} lies between ${round(confidence_95(mu,sigma,5)[0],2)} and ${round(confidence_95(mu,sigma,5)[1],2)}")
  print(f"The average implied share price is ${round(mu,2)}, with a{returns} of {round(result*100,2)}%")
  count, bins, ignored = plt.hist(target_price, 50, density=True)
  plt.title(f"Target Price from {iterations} DCFs on {ticker}")
  plt.xlabel("Target Price (USD)")
  plt.ylabel("Probability")
  plt.axvline(x = mu, color = 'r', linestyle='--')
  plt.plot(bins, 1/(sigma * np.sqrt(2 * np.pi)) *
                np.exp( - (bins - mu)**2 / (2 * sigma**2) ),
          linewidth=2, color='r')
  plt.show()

  return None 

NameError: name 'ticker' is not defined

In [87]:
mc_dcf('msft',0.063,0.05,0.029,0,-0.01,0,0,0,100)

NameError: name 'mc_dcf' is not defined

**Football Field**

In [ ]:
#seaborn boxplot and edit whiskers

**Ownership**

In [ ]:
ticker = 'msft'

In [ ]:
holders = pd.DataFrame(si.get_holders(ticker)['Major Holders'])
format_table(holders)

In [ ]:
institutions = pd.DataFrame(si.get_holders(ticker)['Direct Holders (Forms 3 and 4)'])
format_table(institutions)

#function that flags for activist investors

**Earnings**

In [ ]:
ticker = 'msft'

In [ ]:
earnings = pd.DataFrame(si.get_analysts_info(ticker)['Earnings History'])
print(f"Earnings per share of {ticker}")
format_table(earnings)

**Multiples**

In [ ]:
format_table(si.get_stats_valuation(ticker))

**Comps**

In [ ]:
sp_500 = si.tickers_sp500()

In [ ]:
def comps(ticker):
  comp = []
  mkt_cap = string_to_int(si.get_quote_table(ticker)['Market Cap'])
  for i in range(len(sp_500)):
    if (string_to_int(si.get_quote_table(sp_500[i])['Market Cap'])/mkt_cap <= 1.3 and string_to_int(si.get_quote_table(sp_500[i])['Market Cap'])/mkt_cap >= 0.7):
      comp.append(sp_500[i])

In [ ]:
import requests
import json
from collections import Counter
import pandas as pd
#import matplotlib
#from pylab import title, figure, xlabel, ylabel, xticks, bar, legend, axis, savefig
#from fpdf import FPDF
from bs4 import BeautifulSoup as bs
import yahoo_fin
from yahoo_fin import news
from yahoo_fin import stock_info as si
from pprint import pprint

# virtualenv -p /usr/local/bin/python discord_env
# to make virtual env with python 3.9

# 4a4fb18ec5mshf8dcd4eaab49c67p10ca58jsn07e15446cd38

# spare key:
# a65004e0bamshf40ece7248a25bbp1fe136jsn477cd500d0d0

headers = {
    'x-rapidapi-key': "a65004e0bamshf40ece7248a25bbp1fe136jsn477cd500d0d0",
    'x-rapidapi-host': "yahoo-finance15.p.rapidapi.com"
}


def fetch_comps(ticker, metric):

    response = news.get_yf_rss(ticker)
    ticker = ticker.upper()

    # pprint(response)

    if len(response) == 0:
        print("ERROR: Ticker not found")
    else:
        ticker_list = []
        for article in response:

            link = article['link']
            page = requests.get(link)
            soup = bs(page.text, 'html.parser')
            duplicate_list = [x.text for x in soup.find_all(
                'div', attrs={"class": "xray-pill-label"})]
            final_list = list(set(duplicate_list))

            for item in final_list:
                ticker_list.append(item)

        c = Counter(ticker_list)

        ticker_profile_url = "https://finance.yahoo.com/quote/" + \
            ticker + "/profile?p=" + ticker
        page = requests.get(ticker_profile_url)
        soup = bs(page.text, 'html.parser')
        ticker_profile = [x.text for x in soup.find_all(
            'span', attrs={"class": "Fw(600)"})]
        try:
            ticker_sector = ticker_profile[1]
        except IndexError:
            print(
                "Error: ticker does not have a listed sector. Unable to find comparables")
            exit(1)

        for key, value in c.items():
            profile_url = "https://finance.yahoo.com/quote/" + key + "/profile?p=" + key
            page = requests.get(profile_url)
            soup = bs(page.text, 'html.parser')
            curr_profile = [x.text for x in soup.find_all(
                'span', attrs={"class": "Fw(600)"})]
            curr_sector = ""
            try:
                curr_sector = curr_profile[1]
            except IndexError:
                pass

            if ticker_sector == curr_sector:

                peg = fetch_peg(metric, key)
                if peg != 0:
                    print(key + ", PEG: " + str(peg))


def fetch_peg(metric, ticker):

    price = si.get_live_price(ticker)
    price = float(price)
    try:
        earnings = si.get_analysts_info(ticker)['Earnings History']
    except ImportError:
        return 0
    try:
        eps = earnings.iloc[1][4]
        eps = float(eps)
    except KeyError:
        print("Insufficient data to generate PEG analysis")
        exit(1)
    except TypeError:
        print("Insufficient data to generate PEG analysis")
        exit(1)
    try:
        previous_eps = earnings.iloc[1][1]
        previous_eps = float(previous_eps)
    except KeyError:
        print("Insufficient data to generate PEG analysis")
        exit(1)
    except TypeError:
        print("Insufficient data to generate PEG analysis")
        exit(1)

    annual_eps_growth = eps/previous_eps
    annual_eps_growth = annual_eps_growth - 1
    annual_eps_growth = annual_eps_growth*100

    numerator = price/eps

    denominator = annual_eps_growth
    return (numerator/denominator)


def main():
    ticker = input("Enter a ticker: ")
    metric = input("What method would you like to be evaluated: ")
    fetch_comps(ticker, metric)


main()